In [1]:
"""
SOPRANO: a Python library for generation, manipulation and analysis of large batches of crystalline structures
by Simone Sturniolo
      _
    /|_|\ 
   / / \ \
  /_/   \_\
  \ \   / /
   \ \_/ /
    \|_|/
    
Developed within the CCP-NC project. Copyright STFC 2016


TUTORIAL 1 - Basic concepts: using AtomsCollection objects

"""

print

In [2]:
# Basic imports
import os, sys
sys.path.insert(0, os.path.abspath('..')) # This to add the Soprano path to the PYTHONPATH
                                          # so we can load it without installing it

In [3]:
# Other useful imports

import glob

import numpy as np

import ase
from ase import io as ase_io

from soprano.collection import AtomsCollection

In [18]:
# TO BE REMOVED AFTER COMPLETION

bccfe = ase.Atoms('FeFeFeFe',
                  scaled_positions=[[0, 0, 0], [0.25, 0.5, 0.5], [0.5, 0, 0], [0.75, 0.5, 0.5]],
                  cell=[2.9*2, 2.9, 2.9])
fccfe = ase.Atoms('FeFeFeFe',
                  scaled_positions=[[0, 0, 0], [0.5, 0.5, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5]],
                  cell=[3.59]*3)

from soprano.collection.generate import rattleGen

rgbcc = rattleGen(bccfe, amplitude=0.1)
rgfcc = rattleGen(fccfe, amplitude=0.1)

for i in range(10):
    rgrnd = np.random.choice([rgbcc, rgfcc])
    s = next(rgrnd)
    s.set_cell(s.get_cell() + np.random.random((3,3))*0.05-0.025)
    ase_io.write('tutorial_data/struct_{0}.cif'.format(i+1), s)

In [4]:
"""
1 - LOADING STRUCTURES

Soprano can handle multiple structure loading into a single AtomsCollection object.
The structures are loaded singularly as ASE (Atomic Simulation Environment) Atoms objects.
"""

# List all files in the tutorial directory
cifs = glob.glob('tutorial_data/struct*.cif')

aColl = AtomsCollection(cifs, progress=True) # "progress" means we will visualize a loading bar

Loading collection...
Loading: [████████████████████] |
Loaded 10 structures


In [20]:
"""
2 - HANDLING COLLECTIONS

Collections are a convenient way of manipulating multiple structures. They allow for many operations that act
collectively on all Atoms objects, or return values from them all at once.
"""

help(AtomsCollection)

Help on class AtomsCollection in module soprano.collection.collection:

class AtomsCollection(__builtin__.object)
 |  AtomsCollection object.
 |  
 |  An AtomsCollection represents a group of ASE Atoms objects.
 |  It handles them together, can perform mass operations on them, and stores
 |  arrays of informations related to them.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      Addition of two collections brings a merging
 |  
 |  __deepcopy__(self, memodict={})
 |      Protects against problems with infinite recursion in AllCaller
 |  
 |  __getitem__(self, indices)
 |      Allow sophisticated slicing
 |  
 |  __iadd__(self, other)
 |  
 |  __init__(self, structures=[], info={}, cell_reduce=False, progress=False)
 |      Initialize the AtomsCollection
 |      
 |      | Args:
 |      |    structures (list[str] or list[ase.Atoms]): list of file names or
 |      |                                               Atoms that will form
 |      |                           

In [14]:
# To access an individual structure, one can simply use indexing:
a0 = aColl.structures[0]
print '---- struct_0.cif positions ----\n'
print a0.get_positions(), '\n\n'

# All properties and methods of Atoms objects are available on an entire collection too, by using
# the meta-element 'all'

print '---- all struct_*.cif positions----\n'
print aColl.all.get_positions(), '\n\n'

print '---- all struct_*.cif info dictionaries----\n'
print aColl.all.info, '\n\n'

---- struct_0.cif positions ----

[[ 0.75793256  0.94751667  0.80647053]
 [ 0.88479699  0.62396811  0.25522664]
 [ 0.09024306  0.59255159  0.03223542]
 [ 0.21161056  0.22623914  0.72407407]] 


---- all struct_*.cif positions----

[[[ 0.75793256  0.94751667  0.80647053]
  [ 0.88479699  0.62396811  0.25522664]
  [ 0.09024306  0.59255159  0.03223542]
  [ 0.21161056  0.22623914  0.72407407]]

 [[ 0.21114542  0.03425193  0.05982127]
  [ 0.5949015   0.29792799  0.36162355]
  [ 0.87110592  0.13072396  0.91271482]
  [ 0.18231481  0.29436869  0.65224727]]

 [[ 0.84595149  0.99287711  0.12329696]
  [ 0.5330563   0.57774029  0.85515617]
  [ 0.9723266   0.0480301   0.15559423]
  [ 0.4294857   0.30131269  0.43110142]]

 [[ 0.08424182  0.95374184  0.71514129]
  [ 0.78576494  0.14243732  0.15553295]
  [ 0.17392238  0.9953966   0.00205546]
  [ 0.04971194  0.30711054  0.63513654]]

 [[ 0.22447854  0.94778182  0.12495251]
  [ 0.45918449  0.33588428  0.78395216]
  [ 0.10894467  0.87567337  0.28785928]
 

In [15]:
# Collections can also be sliced like Numpy arrays for convenience
aColl02 = aColl[0:2]
aColl25 = aColl[2:5]

# Then join them together
aColl05 = aColl02+aColl25

print "---- Collection slice lengths ---- \n"
print "aColl02 = {0}\taColl25 = {1}\taColl05 = {2}\n\n".format(aColl02.length, aColl25.length, aColl05.length)

---- Collection slice lengths ---- 

aColl02 = 2	aColl25 = 3	aColl05 = 5




In [16]:
# Collections can also store "arrays" of data, similarly to Atoms objects in ase
# These arrays' elements are tied each to one structure, and can be used to sort them

arr = range(10, 0, -1) # Let's use this array to reverse the order of a collection

aColl.set_array('reversed_range', arr)

aCollSorted = aColl.sorted_byarray('reversed_range')

print "---- Getting an array from a collection ---- \n"
print "Unsorted: ", aColl.get_array('reversed_range'), "\n"
print "Sorted: ", aCollSorted.get_array('reversed_range'), "\n\n"

# And to make sure
print "---- First vs. last elements ---- \n"
print aColl.structures[0].get_positions(), "\n"
print aCollSorted.structures[-1].get_positions()

---- Getting an array from a collection ---- 

Unsorted:  [10  9  8  7  6  5  4  3  2  1] 

Sorted:  [ 1  2  3  4  5  6  7  8  9 10] 


---- First vs. last elements ---- 

[[ 0.75793256  0.94751667  0.80647053]
 [ 0.88479699  0.62396811  0.25522664]
 [ 0.09024306  0.59255159  0.03223542]
 [ 0.21161056  0.22623914  0.72407407]] 

[[ 0.75793256  0.94751667  0.80647053]
 [ 0.88479699  0.62396811  0.25522664]
 [ 0.09024306  0.59255159  0.03223542]
 [ 0.21161056  0.22623914  0.72407407]]
